In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler

# ===========================
# Paramètres
# ===========================
symbols = ["SOLUSDT"]  # Choisir le symbole à visualiser
model_folder = "."
dataset_file = "dataset_export_1h_4h_1d.csv"
horizon = 10
threshold = 0.01
sample_step = 5  # Échantillonnage pour réduire la densité

# Limites axe des abscisses (changer si besoin)
start_date = "2025-01-01 00:00"
end_date = "2025-02-01 00:00"

# ===========================
# Chargement dataset
# ===========================
df = pd.read_csv(dataset_file)
symbol_mapping = {1: "BTCUSDT", 2: "ETHUSDT", 3: "BNBUSDT", 4: "SOLUSDT"}
df["symbol_label"] = df["symbol_id"].map(symbol_mapping)
data = df[df["symbol_label"] == symbols[0]].copy()

# Target
data["future_return"] = data["close"].shift(-horizon) / data["close"] - 1
data["target"] = np.where(data["future_return"] > threshold, 1,
                          np.where(data["future_return"] < -threshold, 0, np.nan))
data.dropna(subset=["target"], inplace=True)

# Features
def compute_RSI(close, period=14):
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

def compute_indicators(df):
    df = df.copy()
    df["RSI"] = compute_RSI(df["close"])
    df["MA_10"] = df["close"].rolling(10).mean()
    df["MA_50"] = df["close"].rolling(50).mean()
    df["STD_20"] = df["close"].rolling(20).std()
    df["RET"] = df["close"].pct_change()
    df["VOLAT"] = df["RET"].rolling(10).std()
    for lag in [1,2,3]:
        df[f"close_lag{lag}"] = df["close"].shift(lag)
        df[f"volume_lag{lag}"] = df["volume"].shift(lag)
    df.dropna(inplace=True)
    return df

data_feat = compute_indicators(data)
features = ["open","high","low","close","volume","RSI","MA_10","MA_50","STD_20","RET","VOLAT",
            "close_lag1","close_lag2","close_lag3","volume_lag1","volume_lag2","volume_lag3"]
X = data_feat[features]
y = data_feat["target"]

# Normalisation et modèle
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
model_file = f"{model_folder}/{symbols[0]}_XGBoost_OPT.pkl"
model = joblib.load(model_file)
data_feat["prediction"] = model.predict(X_scaled)

# Conversion datetime et échantillonnage
data_feat["datetime"] = pd.to_datetime(data_feat["close_time"])
df_plot = data_feat.iloc[::sample_step]

# ===========================
# Visualisation
# ===========================
plt.figure(figsize=(18,8))

# Signal réel
buy_idx = df_plot[df_plot["target"] == 1].index
sell_idx = df_plot[df_plot["target"] == 0].index
plt.scatter(df_plot.loc[buy_idx, "datetime"], df_plot.loc[buy_idx, "close"],
            label="Signal ACHAT réel", marker="^", s=60, color="green")
plt.scatter(df_plot.loc[sell_idx, "datetime"], df_plot.loc[sell_idx, "close"],
            label="Signal VENTE réel", marker="v", s=60, color="red")

# Signal modèle
buy_pred = df_plot[df_plot["prediction"] == 1].index
sell_pred = df_plot[df_plot["prediction"] == 0].index
plt.scatter(df_plot.loc[buy_pred, "datetime"], df_plot.loc[buy_pred, "close"],
            label="Signal ACHAT modèle", marker="^", s=80, facecolors='none', edgecolors='green')
plt.scatter(df_plot.loc[sell_pred, "datetime"], df_plot.loc[sell_pred, "close"],
            label="Signal VENTE modèle", marker="v", s=80, facecolors='none', edgecolors='red')

# Limites axe des abscisses
plt.xlim(pd.to_datetime(start_date), pd.to_datetime(end_date))

# Axe date
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M"))
plt.gcf().autofmt_xdate()
plt.title(f"{symbols[0]} : Signal réel vs modèle XGBoost", fontsize=16)
plt.xlabel("Temps")
plt.ylabel("Prix")
plt.legend()
plt.grid(True)
plt.show()
